In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("Dataset_Datathon_Fraud_Detection_28122022.xlsx")

In [3]:
df.describe

<bound method NDFrame.describe of            customer_id     event_dttm_Deli    event_type_nm  \
0        customer_id_0 2022-10-01 10:51:36  CHANGE_PASSWORD   
1        customer_id_0 2022-10-01 11:15:21   SESSION_SIGNIN   
2        customer_id_0 2022-10-01 11:31:55   SESSION_SIGNIN   
3        customer_id_0 2022-10-01 11:45:42   SESSION_SIGNIN   
4        customer_id_0 2022-10-03 10:28:24          DEPOSIT   
...                ...                 ...              ...   
10964  customer_id_199 2022-10-29 01:22:11   SESSION_SIGNIN   
10965  customer_id_199 2022-10-29 12:12:53   SESSION_SIGNIN   
10966  customer_id_199 2022-10-29 12:13:27          PAYMENT   
10967  customer_id_199 2022-10-29 15:26:58   SESSION_SIGNIN   
10968  customer_id_199 2022-10-31 21:55:16          PAYMENT   

      client_defined_event_type_nm               event_desc  \
0                            RENEW  Login password recovery   
1                   SESSION_SIGNIN                    Login   
2                   

In [4]:
df.columns

Index(['customer_id', 'event_dttm_Deli', 'event_type_nm',
       'client_defined_event_type_nm', 'event_desc',
       'operaton_amt_100 (rupees)', 'active phone call',
       'remote desktop protokol', 'emulator', 'f', 'device_id', 'os',
       'device_model', 'screensize', 'agentconnectiontype', 'screen_hash',
       'device for user', 'reciever', 'password change', 'fast change of ip',
       'ip not equal to previous'],
      dtype='object')

In [5]:
df["ip not equal to previous"]

0        IP differes
1        IP differes
2        IP differes
3        IP differes
4                NaN
            ...     
10964    IP differes
10965    IP differes
10966    IP differes
10967    IP differes
10968            NaN
Name: ip not equal to previous, Length: 10969, dtype: object

In [6]:
df.sort_values(['customer_id', 'event_dttm_Deli'], inplace=True)
df['new_session'] = (df['event_type_nm'] == 'SESSION_SIGNIN').astype(int)
df['session_num'] = df.groupby('customer_id')['new_session'].cumsum()
df.drop(['new_session'], axis=1, inplace=True)

In [7]:
df['pre_auth_flag'] = df['event_desc'].fillna('').str.lower().str.contains('authorization')

In [32]:
df['pre_auth_amount'] = (df['operaton_amt_100 (rupees)'][df['pre_auth_flag']])

In [35]:
df['pre_auth_amount'] = df['pre_auth_amount'].fillna(0)

In [36]:
df.columns

Index(['customer_id', 'event_dttm_Deli', 'event_type_nm',
       'client_defined_event_type_nm', 'event_desc',
       'operaton_amt_100 (rupees)', 'active phone call',
       'remote desktop protokol', 'emulator', 'f', 'device_id', 'os',
       'device_model', 'screensize', 'agentconnectiontype', 'screen_hash',
       'device for user', 'reciever', 'password change', 'fast change of ip',
       'ip not equal to previous', 'session_num', 'pre_auth_flag',
       'pre_auth_amount'],
      dtype='object')

In [11]:
from scipy import stats

In [12]:
import numpy as np

In [38]:
session_df = df.fillna('').groupby(['customer_id','session_num']).agg(
    timestamp_min = ('event_dttm_Deli',min),timestamp_max = ('event_dttm_Deli',max),
    event_desc = ('event_desc',lambda x: ','.join(x)),
    amount_sum = ('operaton_amt_100 (rupees)',lambda x: sum(x)),
    curr_device = ('device_model',lambda x: max(x)),#value_counts().index[0] if x.value_counts().index else '')
    curr_location = ('f','max'),
    pre_auth_count = ('pre_auth_flag','sum'),
    pre_auth_amt = ('pre_auth_amount','sum')
#     payment_cnt = ('event_desc',lambda x: sum(('Purchase' in x) | ('Payment' in x)))
)

In [39]:
session_df

timestamp_min       timestamp_max  \
customer_id    session_num                                           
customer_id_0  0           2022-10-01 10:51:36 2022-10-01 10:51:36   
               1           2022-10-01 11:15:21 2022-10-01 11:15:21   
               2           2022-10-01 11:31:55 2022-10-01 11:31:55   
               3           2022-10-01 11:45:42 2022-10-03 10:28:24   
               4           2022-10-03 10:29:37 2022-10-03 10:29:37   
...                                        ...                 ...   
customer_id_99 20          2022-11-18 18:35:22 2022-11-18 20:14:57   
               21          2022-11-18 20:20:19 2022-11-19 18:32:44   
               22          2022-11-20 00:08:33 2022-11-20 00:08:33   
               23          2022-11-20 15:36:10 2022-11-20 15:47:50   
               24          2022-11-20 20:11:08 2022-11-20 20:11:08   

                                                                   event_desc  \
customer_id    session_num                                                      
customer_id_0  0                                      Login password recovery   
               1                                                        Login   
               2                                                        Login   
               3                                          Login,Payment order   
               4                                                        Login   
...                                                                       ...   
customer_id_99 20                            Login,Purchase,Purchase,Purchase   
               21           Login,Payment between Telebank clients (Person...   
               22                                                       Login   
               23                                              Login,Purchase   
               24                                                       Login   

                            amount_sum curr_device  curr_location  \
customer_id    session_num                                          
customer_id_0  0                  0.00              St Petersburg   
               1                  0.00              St Petersburg   
               2                  0.00              St Petersburg   
               3             116820.00      iPhone  St Petersburg   
               4                  0.00              St Petersburg   
...                                ...         ...            ...   
customer_id_99 20             17223.28      iPhone         Moscow   
               21             78962.06      iPhone         Moscow   
               22                 0.00      iPhone         Moscow   
               23              1294.46      iPhone         Moscow   
               24                 0.00      iPhone         Moscow   

                            pre_auth_count  pre_auth_amt  
customer_id    session_num                                
customer_id_0  0                       0.0           0.0  
               1                       0.0           0.0  
               2                       0.0           0.0  
               3                       0.0           0.0  
               4                       0.0           0.0  
...                                    ...           ...  
customer_id_99 20                      0.0           0.0  
               21                      0.0           0.0  
               22                      0.0           0.0  
               23                      0.0           0.0  
               24                      0.0           0.0  

[2746 rows x 8 columns]

In [ ]:
session_df['duration'] = (session_df['timestamp_max'] - session_df['timestamp_min']).dt.total_seconds()

In [ ]:
session_df

In [ ]:
session_df['event_desc'] = session_df['event_desc'].str.lower()

In [ ]:
session_df['event_desc_flag'] = session_df['event_desc'].str.contains('password') & (session_df['event_desc'].str.contains('payment') | session_df['event_desc'].str.contains('purchase'))

In [ ]:
session_df

In [ ]:
from statistics import mean,mode,stdev

In [ ]:
customer_df = df.fillna('').groupby(['customer_id']).agg(
    amount_stdev = ('operaton_amt_100 (rupees)',stdev),amount_mean = ('operaton_amt_100 (rupees)','mean'),device = ('device_model',max),operating_sys = ('os','max'),location = ('f','max') 
)

In [ ]:
customer_df

In [ ]:
df_merged = pd.merge(session_df, customer_df, left_on='customer_id', right_on='customer_id', how='left')#.drop('id1', axis=1)

In [ ]:
df_merged

In [ ]:
df_merged['amount_flag'] = df_merged['amount_sum'] > df_merged['amount_stdev'] 

In [ ]:
df_merged['device_flag'] = df_merged['curr_device'] != df_merged['device'] 

In [ ]:
df_merged['location_flag'] = df_merged['curr_location'] != df_merged['location'] 

In [ ]:
df_merged

In [ ]:
df_merged.to_csv('final_merged.csv')